In [12]:
import os
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.retrievers import WikipediaRetriever
from pathlib import Path

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


def make_file(keyword, content):
    # 저장할 경로 설정
    save_text_path = Path("./.cache/research_files")
    save_text_path.mkdir(parents=True, exist_ok=True)  # 디렉토리 생성 (존재하지 않으면)

    # 파일 저장 경로 설정
    file_path = "./.cache/research_files/" + keyword + ".txt"

    # 파일 저장 (텍스트 모드)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)


class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for. Example query: Research about the nomad"
    )


class DDGSearchTool(BaseTool):
    name = "DDGSearchTool"
    description = """
    Use this tool to find the full html content of a search result for a keyword.
    It takes a query as an argument.
    
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class WPSearchTool(BaseTool):
    name = "WPSearchTool"
    description = """
     Use this tool to find the full html content of a search result for a keyword.
    It takes a query as an argument.
    
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        retriever = WikipediaRetriever(top_k_results=5)
        docs = retriever.get_relevant_documents(query)
        return "\n\n".join(doc.page_content for doc in docs)


class TextFileMakerArgsSchema(BaseModel):
    keyword: str = Field(
        description="The string of a keyword.",
    )
    content: str = Field(
        description="The full string of a content.",
    )


class TextFileMakerTool(BaseTool):
    name = "TextFileMakerTool"
    description = """
    Use this to make a text file for a content.
    You should enter a keyword and a full string of a content.
    """
    args_schema: Type[TextFileMakerArgsSchema] = TextFileMakerArgsSchema

    def _run(self, keyword, content):
        make_file(keyword, content)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        DDGSearchTool(),
        WPSearchTool(),
        TextFileMakerTool(),
    ],
)

prompt = "Research about the XZ backdoor using two search tools:DuckDuckGo and Wikipedia. And save results as a .txt file, with a keyword as a file name. If there's two results from each search tool, split each result adding a search tool name on the top of a result, using '[]' brakets."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DDGSearchTool` with `{'query': 'XZ backdoor'}`


What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ... That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. A backdoor was planted in the XZ utils package, a popular open-source compression utility for Linux/UNIX systems, by a threat actor named JiaT75. The backdoor exploited the SSH protocol to gain remote access and control over affected devices. A malicious backdoor was found in a compression tool that made its way into widely used Linux distributions, including Red Hat and Debian. The backdoor, introduced by a developer named JiaT75, interferes with SSH authentication and allows unauthorized access to the

/Users/sask/fullstack-gpt/env/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/sask/fullstack-gpt/env/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.


== Background ==
Microsoft employee and PostgreSQL developer Andres Freund reported the backdoor after investigating a performance regression in Debian Sid. Freund noticed that SSH connec

{'input': "Research about the XZ backdoor using two search tools:DuckDuckGo and Wikipedia. And save results as a .txt file, with a keyword as a file name. If there's two results from each search tool, split each result adding a search tool name on the top of a result, using '[]' brakets.",
 'output': 'The research on the XZ backdoor has been completed using both DuckDuckGo and Wikipedia. The results have been saved as a text file named "XZ_backdoor.txt". Here are the contents of the file:\n\n```\n[DuckDuckGo]\nWhat Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ... That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. A backdoor was planted in the XZ utils package, a popular open-source compression utility for